In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.merge import concatenate, Concatenate
from keras.regularizers import l2
import numpy as np
import keract

In [ ]:
# example of progressively loading images from file
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16

dir_path = 'train_set/'
# numbers are based on mode, varified with histogram
img_width, img_height = 224, 224

# create generator
datagen = ImageDataGenerator(validation_split=0.2)
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory(dir_path, 
                                       batch_size=64,
                                       target_size=(img_width, img_height),
                                       subset='training')
val_it = datagen.flow_from_directory(dir_path,
                                     batch_size=64,
                                     target_size=(img_width, img_height),
                                     subset='validation')
# confirm the iterator works
batchX, batchy = train_it.next()
val_batchX, val_batchy  = val_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
image_base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet', classes=4)

In [ ]:
image_model = Sequential()
image_model.add(image_base_model)
image_model.add(Flatten(input_shape=image_base_model.output_shape[1:], name='top_flatter'))
image_model.add(Dense(256, activation='relu', name='top_relu', kernel_regularizer=l2(1e-5)))
image_model.add(Dropout(0.5))
image_model.add(Dense(256, activation='relu', name='top_relu2', kernel_regularizer=l2(1e-5)))
image_model.add(Dropout(0.5))
image_model.add(Dense(4, activation='sigmoid', name='top_sigmoid', kernel_regularizer=l2(1e-5)))

for layer in image_base_model.layers:
    layer.trainable = False


image_model.compile(optimizer="adam", metrics=["accuracy"], loss="categorical_crossentropy")

In [ ]:
image_model.summary()

In [ ]:
image_model.fit_generator(
    train_it,
    epochs=5,
    validation_data=val_it,
)

In [ ]:
activations = []
for im in []

In [ ]:
co = np.concatenate((activations['top_relu2/Relu:0'][0], text_features[0].toarray()[0]), axis=0)

In [ ]:
tfidf = TfidfVectorizer(analyzer="char", max_df=0.4, max_features=5000, min_df=3, ngram_range=(1, 3), use_idf=True)

In [ ]:
test_sentences = pd.read_csv("test_data.csv").fillna("none").Corrected_text

In [ ]:
text_features = tfidf.fit_transform(test_sentences)

In [ ]:
text_features.toarray()

In [ ]:
image_predictions = [i]

In [ ]:
from keras.preprocessing import image
from keras.applications.nasnet import preprocess_input, decode_predictions

In [ ]:
img = image.load_img("test_set/test_1.jpg", target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [ ]:
def preprocess_image(x):
    img = image.load_img(x, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    

In [ ]:
def make_final_model(model, vec_size, n_classes=4, do=0.5, l2_strength=1e-5):
    """
    model : The VGG conv only part with loaded weights
    vec_size : The size of the vectorized text vector coming from the bag of words 
    n_classes : How many classes are you trying to classify ? 
    do : 0.5 Dropout probability
    l2_strenght : The L2 regularization strength
    
    output : The full model that takes images of size (224, 224) and an additional vector
    of size vec_size as input
    """
    
    ### top_aux_model takes the vectorized text as input
    top_aux_model = Sequential()
    top_aux_model.add(Dense(vec_size, input_shape=(vec_size,), name='aux_input'))

#     ### top_model takes output from VGG conv and then adds 2 hidden layers
#     top_model = Sequential()
#     top_model.add(Flatten(input_shape=model.output_shape[1:], name='top_flatter'))
#     top_model.add(Dense(256, activation='relu', name='top_relu', W_regularizer=l2(l2_strength)))
#     top_model.add(Dropout(do))
#     top_model.add(Dense(256, activation='softmax', name='top_softmax', W_regularizer=l2(l2_strength)))
#     ### this is than added to the VGG conv-model
#     model.add(top_model)
    
    ### here we merge 'model' that creates features from images with 'top_aux_model'
    ### that are the bag of words features extracted from the text. 
    merged = Concatenate([model, top_aux_model])

    ### final_model takes the combined feature vectors and add a sofmax classifier to it
    final_model = Sequential()
    final_model.add(merged)
    final_model.add(Dropout(do))
    final_model.add(Dense(n_classes, activation='softmax'))

    return final_model

In [ ]:
fmodel = make_final_model(image_model, 4403)